In [1]:
import pandas as pd
import numpy as np
import json

# 키, 몸무게, 레이블이 적힌 CSV 파일 읽어 들이기
csv = pd.read_csv("bmi.csv")
# 레이블을 배열로 변환하기
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))
# 데이터 정규화
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100
print(csv)


       height  weight   label  label_pat
0       0.710    0.62     fat  [0, 0, 1]
1       0.710    0.73     fat  [0, 0, 1]
2       0.885    0.61  normal  [0, 1, 0]
3       0.935    0.48    thin  [1, 0, 0]
4       0.765    0.60     fat  [0, 0, 1]
...       ...     ...     ...        ...
19995   0.610    0.58     fat  [0, 0, 1]
19996   0.965    0.69  normal  [0, 1, 0]
19997   0.965    0.37    thin  [1, 0, 0]
19998   0.975    0.51    thin  [1, 0, 0]
19999   0.815    0.67  normal  [0, 1, 0]

[20000 rows x 4 columns]


In [2]:
step=0
rows = csv[step*10:(step+1)*10]
x_pat = rows[["weight","height"]]
y_ans = rows["label_pat"]

csv[0:0]

,height,weight,label,label_pat


In [5]:
import tensorflow.compat.v1 as tf

tf.compat.v1.disable_eager_execution()


In [6]:
step = 0
rows = csv[1+step*10:(step+1)*10]
x_pat = rows[["weight","height"]]
y_ans = list(rows["label_pat"])

x = tf.placeholder(tf.float32, [None, 2], name="x")
y_ = tf.placeholder(tf.float32, [None, 3], name="y_")

sess = tf.Session()
#sess.run(x, feed_dict={x: x_pat})
sess.run(y_, feed_dict={y_: y_ans})




Metal device set to: Apple M2


2023-02-14 16:09:12.978000: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-14 16:09:12.978046: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [7]:
# 플레이스홀더 선언
x = tf.placeholder(tf.float32, [None, 2], name="x") # 키와 몸무게 데이터 넣기
y_ = tf.placeholder(tf.float32, [None, 3]) # 정답 레이블 넣기

# 변수 선언
W = tf.Variable(tf.random_uniform([2, 3], -1.0, 1.0)); # 가중치
b = tf.Variable(tf.zeros([3])); # 바이어스
# 소프트맥스 회귀 정의
y = tf.nn.softmax(tf.matmul(x, W) + b)

# 모델 훈련
cross_entropy = -tf.reduce_sum(y_ * tf.log(tf.clip_by_value(y, 1e-10,1.0)))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

# 정답률을 얻기 위해 테스트 데이터 준비
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])
# 정답률 구하기
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(predict, "float"))

# 세션 시작
sess = tf.Session()
sess.run(tf.initialize_all_variables()) #변수 초기화
# 학습
for step in range(5000):
    i = (step * 100) % 1000
    rows = csv[1 + i : 1 + i + 100]
    x_pat = rows[["weight","height"]]
    y_ans = list(rows["label_pat"])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict=fd)
    if step % 100 == 0:
        e = sess.run(cross_entropy, feed_dict=fd)
        a = sess.run(acc, feed_dict={x: test_pat, y_: test_ans})
        print("step=", step, "ce=", e, "acc=", a)
        

Instructions for updating:
Use `tf.global_variables_initializer` instead.


2023-02-14 16:09:16.193824: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-14 16:09:16.193847: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-02-14 16:09:16.195552: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-02-14 16:09:16.196495: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-14 16:09:16.196793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-14 16:09:16.424592: I tensorflow/core/grappler/optimizers/custom_graph_

step= 0 ce= 111.40705 acc= 0.36839998
step= 100 ce= 82.71759 acc= 0.6872
step= 200 ce= 70.42372 acc= 0.7664
step= 300 ce= 63.31199 acc= 0.81439996
step= 400 ce= 58.480614 acc= 0.84459996
step= 500 ce= 54.887173 acc= 0.8696
step= 600 ce= 52.05828 acc= 0.8876
step= 700 ce= 49.74348 acc= 0.9004
step= 800 ce= 47.795727 acc= 0.90919995
step= 900 ce= 46.12197 acc= 0.91779995
step= 1000 ce= 44.659843 acc= 0.9262
step= 1100 ce= 43.365685 acc= 0.9338
step= 1200 ce= 42.207794 acc= 0.9388
step= 1300 ce= 41.162464 acc= 0.94299996
step= 1400 ce= 40.21154 acc= 0.9472
step= 1500 ce= 39.34082 acc= 0.949
step= 1600 ce= 38.53899 acc= 0.95299995
step= 1700 ce= 37.796913 acc= 0.95379996
step= 1800 ce= 37.10711 acc= 0.95659995
step= 1900 ce= 36.463364 acc= 0.9582
step= 2000 ce= 35.86049 acc= 0.96019995
step= 2100 ce= 35.29409 acc= 0.96239996
step= 2200 ce= 34.76043 acc= 0.963
step= 2300 ce= 34.256287 acc= 0.964
step= 2400 ce= 33.7789 acc= 0.96459997
step= 2500 ce= 33.325848 acc= 0.965
step= 2600 ce= 32.895

In [8]:
# 데이터 예측
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(predict, "float"))
r = sess.run(acc, feed_dict={x: test_pat, y_:test_ans})
print(r)

0.9478


2023-02-14 16:09:25.057944: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [9]:
test = tf.argmax(y, 1)
r = sess.run(test, feed_dict={x: [[70/100, 170/200]]})
print(r)
                                                          

2023-02-14 16:09:29.666842: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


[1]
